In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('image', cmap='gray')
import cv2
import os
import math
import tensorflow as tf
from tensorflow.keras import backend as K 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (BatchNormalization, Conv2D, Conv2DTranspose, LeakyReLU, Activation,
                                    Flatten, Dense, Reshape, Dropout, Add, Input)
from tensorflow_examples.models.pix2pix import pix2pix
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from sklearn.metrics import roc_curve, roc_auc_score
import random
import pickle
import sys

from random import randint, uniform
from numpy.random import normal, uniform
from skimage.util import random_noise
from skimage.filters import threshold_otsu
from skimage.draw import ellipse_perimeter, disk
from scipy.interpolate import interp1d 
from scipy.ndimage import gaussian_filter

## Set up model defs and pipeline processes

In [ ]:
# TODO: make this more OOP-like. Also add uncertainty option for mobilenet version
class AEUnet:
    @staticmethod
    def collin(dim=256,depth=1,filters=(16,32,64,128,256),latentDepth=512,MCDropout=False):
        '''
        dim: dim of square input image
        depth: number of channels in input image
        filters: tuple for set of convolution filters, defaulted to paper implementation.
        latentDepth: depth of latent layer
        '''
        inputShape=(dim,dim,depth)
        chanDim=-1 #channel dimension (-1) applies batch norm for each layer (or depth) Unknowns=[mu(untrainable),sigma(untrainable),gamma,beta]*depth

        inputs = Input(shape=inputShape)
        x = inputs

        # generate:conv=>relu=>bn layers
        # additionally, save skip connections and add dropout
        # Dropout layers increase by 0.1 as the encoder gets deeper
        skips = []
        for i, f in enumerate(filters):
            x = Conv2D(f,(5,5),strides=2,padding='same')(x)
            x1 = [x]
            skips.append(x1[0])
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x) 
            x = Dropout(0.1 * i)(x, training=MCDropout)

        # For the UNet build, use a conv layer instead of dense for latent
        latent = Conv2D(latentDepth, (5, 5), strides=2, padding='same', name='latent')(x)
        y = latent
        
        #decode the latent space, incorporating skip layers
        for i, f in reversed(list(enumerate(filters))):
            y = Conv2DTranspose(f,(5,5),strides=2,padding='same')(y)
            y = Add()([skips[i], y])
            y = LeakyReLU(alpha=0.2)(y)
            y = BatchNormalization(axis=chanDim)(y)

        
        # Technically not strictly following the paper, this should be a straight-up upsampling
        y = Conv2DTranspose(depth,(5,5),strides=2,padding='same')(y)
        outputs = Activation('sigmoid')(y)
        autoencoder = Model(inputs,outputs,name='autoencoder')

        return autoencoder
    
    # This one flips the order of dropout and batchnorm and adds more dropout to decoder.
    def collin_experimental(dim=256,depth=1,filters=(16,32,64,128,256),latentDepth=512,MCDropout=False):
        '''
        dim: dim of square input image
        depth: number of channels in input image
        filters: tuple for set of convolution filters, defaulted to paper implementation.
        latentDepth: depth of latent layer
        '''
        inputShape=(dim,dim,depth)
        chanDim=-1 #channel dimension (-1) applies batch norm for each layer (or depth) Unknowns=[mu(untrainable),sigma(untrainable),gamma,beta]*depth

        inputs = Input(shape=inputShape)
        x = inputs

        # generate:conv=>relu=>bn layers
        # additionally, save skip connections and add dropout
        # Dropout layers increase by 0.1 as the encoder gets deeper
        skips = []
        for i, f in enumerate(filters):
            x = Conv2D(f,(5,5),strides=2,padding='same')(x)
            x1 = [x]
            skips.append(x1[0])
            x = LeakyReLU(alpha=0.2)(x)
            x = Dropout(0.1 * i)(x, training=MCDropout)
            x = BatchNormalization(axis=chanDim)(x) 

        # For the UNet build, use a conv layer instead of dense for latent
        latent = Conv2D(latentDepth, (5, 5), strides=2, padding='same', name='latent')(x)
        y = latent
        
        #decode the latent space, incorporating skip layers
        for i, f in reversed(list(enumerate(filters))):
            y = Conv2DTranspose(f,(5,5),strides=2,padding='same')(y)
            y = Add()([skips[i], y])
            y = LeakyReLU(alpha=0.2)(y)
            x = Dropout(0.1 * (4-i))(x, training=MCDropout)
            y = BatchNormalization(axis=chanDim)(y)

        
        # Technically not strictly following the paper, this should be a straight-up upsampling
        y = Conv2DTranspose(depth,(5,5),strides=2,padding='same')(y)
        outputs = Activation('sigmoid')(y)
        autoencoder = Model(inputs,outputs,name='autoencoder')

        return autoencoder

    def mobilenet(dim, depth, filters, latentDepth, output_channels=3,trainable=True, MCDropout=False):
        
        # not actually using filters option. Just had it to satisfy the system.
        
        base_model = tf.keras.applications.MobileNetV2(input_shape=[dim, dim, 3], include_top=False)

        # Use the activations of these layers
        layer_names = [
            'block_1_expand_relu',   # 64x64
            'block_3_expand_relu',   # 32x32
            'block_6_expand_relu',   # 16x16
            'block_13_expand_relu',  # 8x8
            'block_16_project',      # 4x4
        ]
        layers = [base_model.get_layer(name).output for name in layer_names]

        # Create the feature extraction model
        down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)
        down_stack.trainable = trainable # From TLP experience, this should be True

        # "Add" layer version
#         up_stack = [
#             pix2pix.upsample(576, 3),  # 4x4 -> 8x8
#             pix2pix.upsample(192, 3),  # 8x8 -> 16x16
#             pix2pix.upsample(144, 3),  # 16x16 -> 32x32
#             pix2pix.upsample(96, 3),   # 32x32 -> 64x64
#         ]

        # "Concat" layer version
        up_stack = [
            pix2pix.upsample(512, 3),  # 4x4 -> 8x8
            pix2pix.upsample(256, 3),  # 8x8 -> 16x16
            pix2pix.upsample(128, 3),  # 16x16 -> 32x32
            pix2pix.upsample(64, 3),   # 32x32 -> 64x64
        ]

        inputs = tf.keras.layers.Input(shape=[dim, dim, 3])
        x = inputs

        # Downsampling through the model
        skips = down_stack(x)
        x = skips[-1]
        skips = reversed(skips[:-1])

        # Latent space
        x = Dense(latentDepth)(x)
        
        # Upsampling and establishing the skip connections
        for up, skip in zip(up_stack, skips):
            x = up(x)
            concat = tf.keras.layers.Concatenate()
            x = concat([x, skip])
            # Add dropout mostly just for uncertainty functionality
            x = Dropout(0.1 * (5-i))(x, training=MCDropout)

        # This is the last layer of the model
        last = tf.keras.layers.Conv2DTranspose(
            output_channels, 3, strides=2,
            padding='same')(x)  #64x64 -> 128x128

        # Modification to make autoencoder
        outputs = Activation('sigmoid')(last)
        autoencoder = Model(inputs,outputs,name='autoencoder')

        return autoencoder
    
def ssim_loss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 1.0))

In [ ]:
#!!! If blur is too high (0.08), then it won't get real defects good
def add_stain(img,
              min_size=10,
              max_size=40,
              min_color=0.,
              max_color=255.,
              cx_range=None,
              cy_range=None,
              irregularity=0.02,
              blur=0.02):
    '''
    Draw an ellipse-like shape 
    INPUT : 
        - img: image to corrupt with an elliptical stain
        - min_size, max_size: bounds for stain size in percentage
        - min_color, max_color: bounds of intensities to sample from (0 - 255)
        - cx_range, cy_range: (optional) tuples to constrain location (in percentage of image) of stain center
        - irregularity: level of irregularity of the ellipse (0: no)
        - blur: blur edges (0: no)
    OUTPUT: 
        - corrupted image
    '''
    
    assert min_size <= max_size, 'min_size must not be greater than max_size'
    assert min_size >= 0 and max_size <= 100, 'invalid sizes. Must be in range [0, 100]'
    assert min_color <= max_color, 'min_color must not be greater than max_color'
    assert max_color >= 0 and max_color <= 255, 'invalid colors, Must be in range[0, 255]'

    color    = randint(min_color, max_color)
    col, row = img.shape[1], img.shape[0]
    rotation = uniform(0, 2*np.pi)
    ax_x = int(randint(min_size, max_size)/2 / 100*col)
    ax_y = int(randint(min_size, max_size)/2 / 100*row)
    
    if cx_range is not None:
        cx_lower = int(cx_range[0] / 100.*col)
        cx_upper = int(cx_range[1] / 100.*col)
    else:
        cx_lower = int(max_size/2 / 100.*col)
        cx_upper = int((100 - max_size/2) / 100.*col)
        
    if cy_range is not None:
        cy_lower = int(cy_range[0] / 100.*row)
        cy_upper = int(cy_range[1] / 100.*row)
    else:
        cy_lower = int(max_size/2 / 100.*row)
        cy_upper = int((100 - max_size/2) / 100.*row)
    
    cx, cy = randint(cx_lower, cx_upper), randint(cy_lower, cy_upper)
    y,x      = ellipse_perimeter(cy, cx, ax_y, ax_x, rotation)
    # Flip x and y because opencv is annoying
    contour  = np.array([[i,j] for i,j in zip(x,y)])
    # Change the shape of the ellipse 
    if irregularity > 0: 
        contour = _perturbate_ellipse(contour, cx, cy, (ax_x+ax_y)/2, irregularity)

    mask = np.zeros((row, col)) 
    mask = cv2.drawContours(mask, [contour], -1, 1, -1)

    if blur != 0 : 
        mask = gaussian_filter(mask, max(ax_x,ax_y)*blur)

    rgb_mask     = np.dstack([mask]*3)
    not_modified = np.subtract(np.ones(img.shape), rgb_mask)
    stain        = 255*random_noise(np.zeros(img.shape), mode='gaussian', mean = color/255., var = 0.05/255.)
    result       = np.add( np.multiply(img,not_modified), np.multiply(stain,rgb_mask) ) 

    return result.astype(np.uint8)

'''
Helper functions for stain
'''
def _perturbate_ellipse(contour, cx, cy, diag, irregularity):
    # Keep only some points
    if len(contour) < 20: 
        pts = contour
    else: 
        pts = contour[0::int(len(contour)/20)]

    # Perturbate coordinates
    for idx,pt in enumerate(pts): 
        pts[idx] = [pt[0]+randint(-int(diag*irregularity), int(diag*irregularity)),
                    pt[1]+randint(-int(diag*irregularity),int(diag*irregularity))]
    pts = sorted(pts, key=lambda p: _clockwiseangle(p, cx, cy))
    pts.append([pts[0][0], pts[0][1]])

    # Interpolate between remaining points
    i = np.arange(len(pts))
    interp_i = np.linspace(0, i.max(), 10 * i.max())
    yi = interp1d(i, np.array(pts)[:,0], kind='cubic')(interp_i)
    xi = interp1d(i, np.array(pts)[:,1], kind='cubic')(interp_i) 
 
    return np.array([[int(i),int(j)] for i,j in zip(yi,xi)])

def _clockwiseangle(point, cx, cy):
    refvec = [0 , 1]
    vector = [point[0]-cy, point[1]-cx]
    norm   = math.hypot(vector[0], vector[1])
    # If length is zero there is no angle
    if norm == 0:
        return -math.pi
    normalized = [vector[0]/norm, vector[1]/norm]
    dotprod    = normalized[0]*refvec[0] + normalized[1]*refvec[1] 
    diffprod   = refvec[1]*normalized[0] - refvec[0]*normalized[1] 
    angle      = math.atan2(diffprod, dotprod)
    if angle < 0:
        return 2*math.pi+angle
    return angle

In [ ]:
def get_auc(object_path, autoencoder, method=None, model_function=None):
    
    ground_path  = os.path.join(object_path, 'ground_truth')
    images_path  = os.path.join(object_path, 'test')
    defect_types = os.listdir(ground_path)

    if method == 'uncertainty':
        print('[INFO]: Generating uncertainty model')
        assert model_function is not None, 'Must provide function that autoencoder was created with if using uncertainty method'
        weights = autoencoder.get_weights()
        uncertainty_ae = model_function(MCDropout=True)
        uncertainty_ae.set_weights(weights)
        
    # Calculate the overall AUC separately. Averaging the AUC's for individual categories gives
    # a slightly higher score.
    all_ground  = []
    all_results = []
    for defect_type in defect_types:
        ground_vector = []
        input_images  = []
        masks  = sorted(os.listdir(os.path.join(ground_path, defect_type)))
        images = sorted(os.listdir(os.path.join(images_path, defect_type)))
        for mask in masks:
            full_path = os.path.join(ground_path, defect_type, mask)
            ground = cv2.imread(full_path)[...,0]
            ground = cv2.resize(ground, (DIM, DIM))
            ground[ground > 0] = 1
            ground_vector.append(ground)
        for image in images:
            full_path = os.path.join(images_path, defect_type, image)
            image = cv2.imread(full_path)
            if DEPTH == 1:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (DIM, DIM)).astype('float32') / 255
            input_images.append(image)

        # Predict images
        input_images = np.array(input_images)
        if method =='uncertainty':
            results_list = []
            # Using 30 runs according to the paper
            for i in range(30):
                results = uncertainty_ae(input_images).numpy()
                results = np.squeeze(results, axis=-1)
                results_list.append(results)
            results_vector = np.std(results_list, axis=0)
            plt.imshow(results_vector[0])
            plt.show()
            results_vector = results_vector.flatten()
        else:
            results_vector = autoencoder(input_images).numpy()
            results_vector = np.squeeze(results_vector, axis=-1)
            results_vector = np.abs(results_vector - input_images).flatten()
        ground_vector = np.array(ground_vector).flatten()
        score = roc_auc_score(ground_vector, results_vector)
        print(f'AUC for {defect_type}:', score)

        for i in ground_vector: all_ground.append(i)
        for i in results_vector: all_results.append(i)

    full_score = roc_auc_score(all_ground, all_results)
    print('Average AUC score:', full_score)

## Config cells.

Ideally these should be the only things run differently across different runs but things happen and these aren't exactly backwards compatible lol


### Experiment Log
- I'm not gonna document everything (writing at 1/7/21) before this, too lazy
- Mobilenet with 224 and trainable is bad. Starts overfitting really quick.
- 1/7/21 See what happens if we add a dense latent space to the mobilenet Unet

In [ ]:
# Configs
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 1000
INIT_LR = 1e-3
BS = 8
LOSS = 'mse'
testImg = 'data/mvtec_anomaly_detection/carpet/test/hole/000.png'
modelSavePath = 'outputs/prototype.pb'
historySavePath = 'outputs/prototype_history'
def create_autoencoder():
    return AEUnet.mobilenet(3, DIM)
STAIN_PROPS = [10, 40, 0, 255]

In [ ]:
# Same as Prototype but for only 100 epochs, as 50 was where things were really good in the last
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 100
INIT_LR = 1e-3
BS = 8
LOSS = 'mse'
testImg = 'data/mvtec_anomaly_detection/carpet/test/hole/000.png'
modelSavePath = 'outputs/prototype1.pb'
historySavePath = 'outputs/prototype1_history'
def create_autoencoder():
    return AEUnet.mobilenet(3, DIM)
STAIN_PROPS = [10, 40, 0, 255]

In [ ]:
# Same as Prototype but for only 100 epochs, and less aggro stain colors
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 100
INIT_LR = 1e-3
BS = 8
LOSS = 'mse'
testImg = 'data/mvtec_anomaly_detection/carpet/test/hole/000.png'
modelSavePath = 'outputs/prototype2.pb'
historySavePath = 'outputs/prototype1_history'
def create_autoencoder():
    return AEUnet.mobilenet(3, DIM)
STAIN_PROPS = [10, 40, 150, 200]

In [ ]:
# Collin carpet
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
DEPTH = 1
FILTERS = (DEPTH * 16, DEPTH * 32, DEPTH * 64, DEPTH * 128, DEPTH * 256)
LATENT = DEPTH * 512
EPOCHS = 300
INIT_LR = 1e-3
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/carpet/test/cut/000.png',
    'data/mvtec_anomaly_detection/carpet/test/hole/000.png',
    'data/mvtec_anomaly_detection/carpet/test/metal_contamination/000.png',
    'data/mvtec_anomaly_detection/carpet/test/thread/000.png',
    'data/mvtec_anomaly_detection/carpet/test/color/000.png',
    'data/mvtec_anomaly_detection/carpet/test/good/000.png'
]
modelSavePath = 'outputs/collin_carpet.pb'
historySavePath = 'outputs/collin_history'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Resid 80.4
'''
AUC for metal_contamination: 0.7976807709851567
AUC for cut: 0.7676479321130159
AUC for thread: 0.7704211380532168
AUC for color: 0.9335107927634231
AUC for hole: 0.7793682318671236
Average AUC score: 0.8040516566229404
'''

# Uncert 94.1
'''
AUC for metal_contamination: 0.9754365996420657
AUC for cut: 0.9370223507151894
AUC for thread: 0.8651138925990813
AUC for color: 0.9664916496937275
AUC for hole: 0.9529737797856629
Average AUC score: 0.9408986430342042
'''

In [ ]:
# Collin carpet 3D (12/18)
DIM = 256
DEPTH = 3
FILTERS = (DEPTH * 16, DEPTH * 32, DEPTH * 64, DEPTH * 128, DEPTH * 256)
LATENT = DEPTH * 512
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 300
INIT_LR = 1e-3
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/carpet/test/cut/000.png',
    'data/mvtec_anomaly_detection/carpet/test/hole/000.png',
    'data/mvtec_anomaly_detection/carpet/test/metal_contamination/000.png',
    'data/mvtec_anomaly_detection/carpet/test/thread/000.png',
    'data/mvtec_anomaly_detection/carpet/test/color/000.png',
    'data/mvtec_anomaly_detection/carpet/test/good/000.png'
]
modelSavePath = 'outputs/collin_carpet_3D.pb'
historySavePath = 'outputs/collin_history_3D'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Resid: 86.3
'''
AUC for metal_contamination: 0.9434638215965654
AUC for cut: 0.830731196021751
AUC for thread: 0.8622653518916192
AUC for color: 0.909399036895851
AUC for hole: 0.8564032503952155
Average AUC score: 0.8628954155629363
'''
'''
Uncert: 95.6
AUC for metal_contamination: 0.9878566653632854
AUC for cut: 0.9524887847802426
AUC for thread: 0.9442395637140933
AUC for color: 0.947311419083156
AUC for hole: 0.9645082518587725
Average AUC score: 0.955757885451878


Interestingly, for both resid and uncert, the 1D version outperforms the 3D version.
PROBABLY because the color stains are still grayscale even for the color version.
'''

In [ ]:
# Collin carpet 3D half (12/18)
DIM = 256
DEPTH = 3
FILTERS = (16, 32, 64, 128, 256)
LATENT = 512
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 300
INIT_LR = 1e-3
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/carpet/test/cut/000.png',
    'data/mvtec_anomaly_detection/carpet/test/hole/000.png',
    'data/mvtec_anomaly_detection/carpet/test/metal_contamination/000.png',
    'data/mvtec_anomaly_detection/carpet/test/thread/000.png',
    'data/mvtec_anomaly_detection/carpet/test/color/000.png',
    'data/mvtec_anomaly_detection/carpet/test/good/000.png'
]
modelSavePath = 'outputs/collin_carpet_3D_half.pb'
historySavePath = 'outputs/collin_history_3D_half'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

'''
RESIDUAL:
AUC for metal_contamination: 0.8750591164360945
AUC for cut: 0.7556173948053688
AUC for thread: 0.8248140746826649
AUC for color: 0.9337140297783294
AUC for hole: 0.7727851246013533
Average AUC score: 0.8087412669619461

UNCERT
AUC for metal_contamination: 0.9912070449011537
AUC for cut: 0.9317350012144429
AUC for thread: 0.9164020856059844
AUC for color: 0.9660207290746308
AUC for hole: 0.9474724475126308
Average AUC score: 0.943171346244713

So it looks like if you go 3D without increasing the depth of the model then you only get marginal improvements.
However, full 3D with proportional depth is so freaking slow.
'''

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)
base_model.layers

In [ ]:
# Mobilenet carpet (12/18)
DIM = 256
DEPTH = 3
FILTERS = (16, 32, 64, 128, 256)
LATENT = 512
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 300
INIT_LR = 1e-3
BS = 32
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/carpet/test/cut/000.png',
    'data/mvtec_anomaly_detection/carpet/test/hole/000.png',
    'data/mvtec_anomaly_detection/carpet/test/metal_contamination/000.png',
    'data/mvtec_anomaly_detection/carpet/test/thread/000.png',
    'data/mvtec_anomaly_detection/carpet/test/color/000.png',
    'data/mvtec_anomaly_detection/carpet/test/good/000.png'
]
modelSavePath   = 'outputs/mobilenet_carpet.pb'
historySavePath = 'outputs/mobilenet_history'
modelSavePath2  = 'outputs/mobilenet_carpet_full.pb/'
historySavePath = 'outputs/mobilenet_history_full'
create_autoencoder = AEUnet.mobilenet
STAIN_PROPS = [10, 40, 0, 255]

# will require some manual changes downstream. Need to run second training run unfrozen.

# Before unfreezing:
'''
Resid:
AUC for metal_contamination: 0.7127854998468568
AUC for cut: 0.6054001665842494
AUC for thread: 0.7518795370620136
AUC for color: 0.8750955313057392
AUC for hole: 0.6014542457150654
Average AUC score: 0.6794173591587408

Uncert:
AUC for metal_contamination: 0.5112918904603708
AUC for cut: 0.5037951189852011
AUC for thread: 0.5143294279317485
AUC for color: 0.43974230007975773
AUC for hole: 0.49555054327457027
Average AUC score: 0.4917689652321947

So maybe flip the order of dropout for future?
'''

In [ ]:
# Mobilenet carpet 2 (1/7/21)
DIM = 224
DEPTH = 3
FILTERS = (16, 32, 64, 128, 256)
LATENT = 512
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 300
INIT_LR = 1e-4
BS = 32
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/carpet/test/cut/000.png',
    'data/mvtec_anomaly_detection/carpet/test/hole/000.png',
    'data/mvtec_anomaly_detection/carpet/test/metal_contamination/000.png',
    'data/mvtec_anomaly_detection/carpet/test/thread/000.png',
    'data/mvtec_anomaly_detection/carpet/test/color/000.png',
    'data/mvtec_anomaly_detection/carpet/test/good/000.png'
]
modelSavePath   = 'outputs/mobilenet_carpet_native_res.pb'
historySavePath = 'outputs/mobilenet_history_native_res'
create_autoencoder = AEUnet.mobilenet
STAIN_PROPS = [10, 40, 0, 255]
TRAINABLE=True
LR_SCHEDULE_TYPE = 'cosine'

# So I haven't finished training this one but from the looks of it, changing the starting LR from 1e-3 to 1e-4 
# allowed the loss to get lower, so perhaps this is really just a learning rate problem.
# Also the latent space bit doesn't seem to help. Might as well get rid of it.

# Things to try next:
    # Freeze the layers and train the decoder first. Then unfreeze and train both at an extremely low learning rate.
    # If that doesn't work, go back to full unfreeze and try 1e-4 start with exponential decay
    # If that doesn't work, go back to cosine decay and straight up start with like 1e-5
    

In [ ]:
model = create_autoencoder(depth=DEPTH, filters=FILTERS, latentDepth=LATENT)
model.save(modelSavePath)

In [ ]:
# Collin carpet experimental (12/18)
DIM = 256
DEPTH = 1
FILTERS = (16, 32, 64, 128, 256)
LATENT = 512
DATA_DIR = 'data/mvtec_anomaly_detection/carpet/train/good/'
EPOCHS = 300
INIT_LR = 5e-4
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/carpet/test/cut/000.png',
    'data/mvtec_anomaly_detection/carpet/test/hole/000.png',
    'data/mvtec_anomaly_detection/carpet/test/metal_contamination/000.png',
    'data/mvtec_anomaly_detection/carpet/test/thread/000.png',
    'data/mvtec_anomaly_detection/carpet/test/color/000.png',
    'data/mvtec_anomaly_detection/carpet/test/good/000.png'
]
modelSavePath   = 'outputs/collin_carpet_experimental.pb'
historySavePath = 'outputs/collin_carpet_experimental_history'
create_autoencoder = AEUnet.collin_experimental
STAIN_PROPS = [10, 40, 0, 255]

In [ ]:
# Collin bottle
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/bottle/train/good/'
EPOCHS = 300
INIT_LR = 1e-3
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/bottle/test/broken_large/000.png',
    'data/mvtec_anomaly_detection/bottle/test/broken_large/001.png',
    'data/mvtec_anomaly_detection/bottle/test/broken_small/000.png',
    'data/mvtec_anomaly_detection/bottle/test/broken_small/001.png',
    'data/mvtec_anomaly_detection/bottle/test/contamination/000.png',
    'data/mvtec_anomaly_detection/bottle/test/good/000.png'
]
modelSavePath = 'outputs/collin_bottle.pb'
historySavePath = 'outputs/collin_history/bottle'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

In [ ]:
# Collin toothbrush
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/toothbrush/train/good/'
EPOCHS = 1400
INIT_LR = 1e-3
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/toothbrush/test/defective/000.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/001.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/002.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/003.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/004.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/005.png'
]
modelSavePath = 'outputs/collin_toothbrush.pb'
historySavePath = 'outputs/collin_history/toothbrush'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Achieves 91.1% for both. However, every reconstruction has a noticeable blowout in the top of the brush.

In [ ]:
# Collin toothbrush 2 (12/17)
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/toothbrush/train/good/'
EPOCHS = 1400
INIT_LR = 1e-4
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/toothbrush/test/defective/000.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/001.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/002.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/003.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/004.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/005.png'
]
modelSavePath = 'outputs/collin_toothbrush_2.pb'
historySavePath = 'outputs/collin_history/toothbrush_2'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Quit this one early as there was significant blowout and validation loss bottomed out at around 0.003.

In [ ]:
# Collin toothbrush 3 (12/17)
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/toothbrush/train/good/'
EPOCHS = 1400
INIT_LR = 1e-2
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/toothbrush/test/defective/000.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/001.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/002.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/003.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/004.png',
    'data/mvtec_anomaly_detection/toothbrush/test/defective/005.png'
]
modelSavePath = 'outputs/collin_toothbrush_3.pb'
historySavePath = 'outputs/collin_history/toothbrush_3'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Achieves 91.0% and 94.0% for resid/uncert. No more blowout which is nice, though resid went down from original
# somehow...

# Suggests that key to solving blowout is to increase LR? Might also help get to lower loss. Test on bottle.
# I should record the lowest loss and when that happens lol

# The one for this is like 2.39e-4 for valid loss.

In [ ]:
# Collin bottle 2 (12/17)
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/bottle/train/good/'
EPOCHS = 400
INIT_LR = 1e-2
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/bottle/test/broken_large/000.png',
    'data/mvtec_anomaly_detection/bottle/test/broken_large/001.png',
    'data/mvtec_anomaly_detection/bottle/test/broken_small/000.png',
    'data/mvtec_anomaly_detection/bottle/test/broken_small/001.png',
    'data/mvtec_anomaly_detection/bottle/test/contamination/000.png',
    'data/mvtec_anomaly_detection/bottle/test/good/000.png'
]
modelSavePath = 'outputs/collin_bottle_2.pb'
historySavePath = 'outputs/collin_history_2/bottle'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Resid: 85.9, Uncert: 87.0%
# val loss 2.34e-4 at epoch 389/400

In [ ]:
# Collin grid (12/17)
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/grid/train/good/'
EPOCHS = 320
INIT_LR = 1e-2
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/grid/test/bent/000.png',
    'data/mvtec_anomaly_detection/grid/test/broken/000.png',
    'data/mvtec_anomaly_detection/grid/test/glue/000.png',
    'data/mvtec_anomaly_detection/grid/test/metal_contamination/000.png',
    'data/mvtec_anomaly_detection/grid/test/thread/000.png',
    'data/mvtec_anomaly_detection/grid/test/good/000.png'
]
modelSavePath = 'outputs/collin_grid.pb'
historySavePath = 'outputs/collin_history/grid'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Resid: 88.8, Uncert: 93.9%
# val loss 2.6010e-04 at 303/320

In [ ]:
# Collin capsule (12/17)
DIM = 256
DATA_DIR = 'data/mvtec_anomaly_detection/capsule/train/good/'
EPOCHS = 310
INIT_LR = 1e-2
BS = 16
LOSS = 'mse'
testImgs = [
    'data/mvtec_anomaly_detection/capsule/test/crack/000.png',
    'data/mvtec_anomaly_detection/capsule/test/faulty_imprint/000.png',
    'data/mvtec_anomaly_detection/capsule/test/poke/000.png',
    'data/mvtec_anomaly_detection/capsule/test/scratch/000.png',
    'data/mvtec_anomaly_detection/capsule/test/squeeze/000.png',
    'data/mvtec_anomaly_detection/capsule/test/good/000.png'
]
modelSavePath = 'outputs/collin_capsule.pb'
historySavePath = 'outputs/collin_history/capsule'
create_autoencoder = AEUnet.collin
STAIN_PROPS = [10, 40, 0, 255]

# Resid: 77.3, Uncert: 93.3
# Best val loss: 0.00013910970301367342 at epoch 298

# Likely can improve performance by localizing stains.

## Data input pipeline and images to do intermittent eyeballing

In [ ]:
# data input pipeline
def stain_wrapper(image, target):
    image = image.numpy()
    target = target.numpy()
    image = add_stain(image, STAIN_PROPS[0], STAIN_PROPS[1], STAIN_PROPS[2], STAIN_PROPS[3])
    if DEPTH == 1:
        image = np.expand_dims(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), axis=-1)
        target = np.expand_dims(cv2.cvtColor(target, cv2.COLOR_BGR2GRAY), axis=-1)
    return image.astype('float32') / 255, target.astype('float32') / 255

stain_lambda = lambda x, y: tf.py_function(stain_wrapper, [x, y], ['float32', 'float32'])

imgDir = os.listdir(DATA_DIR)
print(f'[INFO] Reading {len(imgDir)} images into tensors...')
imgs = np.empty((len(imgDir), DIM, DIM, 3), dtype='uint8')
for i, imgName in enumerate(imgDir):
    img = cv2.imread(os.path.join(DATA_DIR, imgName))
    img = cv2.resize(img, (DIM, DIM))
    imgs[i] = img
print(f'[SANITY CHECK] image shape: {imgs[0].shape}')
plt.imshow(imgs[0])
plt.show()
    
print('[INFO] Creating TF Datasets...')
(trainX, valX)=train_test_split(imgs,test_size=0.2,random_state=42)

trainData = tf.data.Dataset.from_tensor_slices((trainX, trainX))\
            .prefetch(tf.data.experimental.AUTOTUNE)\
            .map(stain_lambda)\
            .shuffle(trainX.shape[0])\
            .batch(BS)
            
valData   = tf.data.Dataset.from_tensor_slices((valX, valX))\
            .prefetch(tf.data.experimental.AUTOTUNE)\
            .map(stain_lambda)\
            .batch(BS)
            
print('[INFO] Creating eyeballing set...')
visImages = np.empty((6, DIM, DIM, DEPTH))
for i, path in enumerate(testImgs):
    img = cv2.imread(path)
    # By convention (my convention), last image should be the artificially stained one
    if i == 5:
        img = add_stain(img, STAIN_PROPS[0], STAIN_PROPS[1], STAIN_PROPS[2], STAIN_PROPS[3])
    img = cv2.resize(img, (DIM, DIM))
    if DEPTH == 1:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.expand_dims(img, axis=[0, -1])
    visImages[i] = img.astype('float32') / 255
    plt.imshow(visImages[i])
    plt.show()

## Things to Try After Baseline

### First
- Benchmark AMUnet

REMEMBER to take all the optimizations you used for the mobilenet version and reapply them to base Collin and 3D Collin for fairness. Necessary optimizations include localized staining and 3D colorings.

### Easy
- Adabound optimizer
- More aggro synthetic defect suite
- SSIM loss
- Uncertainty vs Residual
    - Are you serious. Uncertainty is literally just running the model 30 times with the dropout layers active.
    - There is a workaround here https://github.com/keras-team/keras/issues/9412#issuecomment-366487249 but I don't understand it and think I tried it already? Looks like (from the lower comments) people have had trouble with it in new TF. Here is my proposed workflow: Define a new model with MC dropout (aka train=True) and port over the weights.

### Research
- Energy method

### More Benchmarking
- VAE
- L2 Autoencoder
- Try concat layers instead of adding.
- Other UNet models
    - See https://github.com/qubvel/segmentation_models/blob/master/segmentation_models/models/unet.py
- Other segmentation-based

- Might be worthwhile to test how it performs when the area of staining is limited to the object itself.

In [ ]:
LR_SCHEDULE

In [ ]:
# from keras_adabound import AdaBound

print('[INFO] building autoencoder...')
K.clear_session()
autoencoder = create_autoencoder(dim=DIM, depth=DEPTH, filters=FILTERS, latentDepth=LATENT, trainable=TRAINABLE)
# Check if it's there
try:
    if LR_SCHEDULE == 'exponential':
        print('Using exponential decay')
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=INIT_LR,
            decay_steps=EPOCHS * trainX.shape[0] // BS,
            decay_rate=0.96,
            staircase=True)
    elif LR_SCHEDULE == 'cosine':
        print('Using cosine decay')
        lr_schedule = tf.keras.experimental.CosineDecay(
            initial_learning_rate=INIT_LR,
            decay_steps=EPOCHS * trainX.shape[0] // BS)
    else:
        raise NameError('Unrecognized decay scheduler. Defaulting to cosine.')
except NameError:
    lr_schedule = tf.keras.experimental.CosineDecay(
        initial_learning_rate=INIT_LR,
        decay_steps=EPOCHS * trainX.shape[0] // BS)
opt=Adam(learning_rate=lr_schedule)
autoencoder.compile(loss=LOSS, optimizer=opt)


'''
ADABOUND
'''
# WEIGHT_DECAY = 1e-6
# regularizer = tf.keras.regularizers.l2(WEIGHT_DECAY / 2)
# for layer in autoencoder.layers:
#     for attr in ['kernel_regularizer', 'bias_regularizer']:
#         if hasattr(layer, attr) and layer.trainable:
#             setattr(layer, attr, regularizer)
            
# autoencoder.compile(optimizer = AdaBound(lr=INIT_LR, final_lr=0.1),
#               loss = 'mse')
''''''
autoencoder.summary()

# callbacks=[
#     TensorBoard(
#     log_dir=args['output_path'], histogram_freq=0, write_graph=True, write_images=False,
#     update_freq='epoch', profile_batch=2, embeddings_freq=0,
#     embeddings_metadata=None)   
# ]
def show_predictions(model, inputs, preds):
    prediction = model.predict(inputs)
    fig, axes = plt.subplots(1, preds, sharex=True, sharey=True, figsize=(15,15))
    for i in range(preds):
        axes[i].imshow(prediction[i])
    plt.tight_layout()
    plt.show()
    
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if epoch % 50 == 0:
            show_predictions(self.model, visImages[:3], 3)
            show_predictions(self.model, visImages[3:], 3)
            print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

best_loss   = float('inf')
best_epoch  = 0
class RecordBestLossCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global best_loss
        global best_epoch
        if logs['val_loss'] < best_loss:
            best_epoch = epoch
            best_loss  = logs['val_loss']
        

checkpoint_filepath = './outputs/temp_weights/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

print('[INFO] training autoencoder...')
H=autoencoder.fit(
    trainData,
    validation_data=valData,
    epochs=EPOCHS,
    callbacks=[DisplayCallback(), RecordBestLossCallback(), model_checkpoint_callback]
)

print('[INFO] loading best weights and cleanup')
autoencoder.load_weights(checkpoint_filepath)
!rm -rf {checkpoint_filepath}/*

print('[INFO] Saving model...')
autoencoder.save(modelSavePath)

print('Final Prediction:')
show_predictions(autoencoder, visImages[:3], 3)
show_predictions(autoencoder, visImages[3:], 3)

print(f'Best val loss: {best_loss} at epoch {best_epoch}')

In [ ]:
def get_auc(object_path, autoencoder, method=None, model_function=None):
    
    ground_path  = os.path.join(object_path, 'ground_truth')
    images_path  = os.path.join(object_path, 'test')
    defect_types = os.listdir(ground_path)

    if method == 'uncertainty':
        print('[INFO]: Generating uncertainty model')
        assert model_function is not None, 'Must provide function that autoencoder was created with if using uncertainty method'
        weights = autoencoder.get_weights()
        uncertainty_ae = model_function(depth=DEPTH, filters=FILTERS, latentDepth=LATENT, MCDropout=True)
        uncertainty_ae.set_weights(weights)
        
    # Calculate the overall AUC separately. Averaging the AUC's for individual categories gives
    # a slightly higher score.
    all_ground  = []
    all_results = []
    for defect_type in defect_types:
        ground_vector = []
        input_images  = []
        masks  = sorted(os.listdir(os.path.join(ground_path, defect_type)))
        images = sorted(os.listdir(os.path.join(images_path, defect_type)))
        for mask in masks:
            full_path = os.path.join(ground_path, defect_type, mask)
            ground = cv2.imread(full_path)[...,0]
            ground = cv2.resize(ground, (DIM, DIM))
            ground[ground > 0] = 1
            ground_vector.append(ground)
        for image in images:
            full_path = os.path.join(images_path, defect_type, image)
            image = cv2.imread(full_path)
            if DEPTH == 1:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (DIM, DIM)).astype('float32') / 255
            input_images.append(image)

        # Predict images
        input_images = np.array(input_images)
        if method =='uncertainty':
            results_list = []
            # Using 30 runs according to the paper
            for i in range(30):
                results = uncertainty_ae(input_images).numpy()
                results = np.sum(results, axis=-1) / 3
                results_list.append(results)
            results_vector = np.std(results_list, axis=0)
#             plt.imshow(results_vector[0])
#             plt.show()
            results_vector = results_vector.flatten()
        else:
            results_vector = autoencoder(input_images).numpy()
            if DEPTH == 1:
                results_vector = np.squeeze(results_vector, axis=-1)
            results_vector = np.abs(results_vector - input_images)
            if DEPTH > 1:
                results_vector = np.sum(results_vector, axis=-1) / 3
            results_vector = results_vector.flatten()
        ground_vector = np.array(ground_vector).flatten()
        score = roc_auc_score(ground_vector, results_vector)
        print(f'AUC for {defect_type}:', score)

        for i in ground_vector: all_ground.append(i)
        for i in results_vector: all_results.append(i)

    full_score = roc_auc_score(all_ground, all_results)
    print('Average AUC score:', full_score)

In [ ]:
# Backup cell to save model and show final predictions if manually stop early
# show_predictions(autoencoder, visImages[:3], 3)
# show_predictions(autoencoder, visImages[3:], 3)
autoencoder.save(modelSavePath)

In [ ]:
# Backup cell to load model if starting new Jupyter kernel
autoencoder = tf.keras.models.load_model(modelSavePath)

## Evaluate Model

In [ ]:
object_path  = 'data/mvtec_anomaly_detection/carpet/'
get_auc(object_path, autoencoder2, method='resid', model_function=create_autoencoder)

In [ ]:
# from keras_adabound import AdaBound
INIT_LR = 1e-4
# print('[INFO] making trainable edition of autoencoder...')
# weights = autoencoder.get_weights()
# autoencoder2 = create_autoencoder(depth=DEPTH, filters=FILTERS, latentDepth=LATENT, trainable=True, MCDropout=False)
# autoencoder2.set_weights(weights)
# lr_schedule = tf.keras.experimental.CosineDecay(
#     initial_learning_rate=INIT_LR,
#     decay_steps=EPOCHS * trainX.shape[0] // BS)
# opt=Adam(learning_rate=lr_schedule)
# autoencoder2.compile(loss=LOSS, optimizer=opt)


'''
ADABOUND
'''
# WEIGHT_DECAY = 1e-6
# regularizer = tf.keras.regularizers.l2(WEIGHT_DECAY / 2)
# for layer in autoencoder.layers:
#     for attr in ['kernel_regularizer', 'bias_regularizer']:
#         if hasattr(layer, attr) and layer.trainable:
#             setattr(layer, attr, regularizer)
            
# autoencoder.compile(optimizer = AdaBound(lr=INIT_LR, final_lr=0.1),
#               loss = 'mse')
''''''
autoencoder2.summary()

# callbacks=[
#     TensorBoard(
#     log_dir=args['output_path'], histogram_freq=0, write_graph=True, write_images=False,
#     update_freq='epoch', profile_batch=2, embeddings_freq=0,
#     embeddings_metadata=None)   
# ]
def show_predictions(model, inputs, preds):
    prediction = model.predict(inputs)
    fig, axes = plt.subplots(1, preds, sharex=True, sharey=True, figsize=(15,15))
    for i in range(preds):
        axes[i].imshow(prediction[i])
    plt.tight_layout()
    plt.show()
    
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if epoch % 50 == 0:
            show_predictions(self.model, visImages[:3], 3)
            show_predictions(self.model, visImages[3:], 3)
            print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

best_loss   = float('inf')
best_epoch  = 0
class RecordBestLossCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global best_loss
        global best_epoch
        if logs['val_loss'] < best_loss:
            best_epoch = epoch
            best_loss  = logs['val_loss']
        

checkpoint_filepath = './outputs/temp_weights/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

print('[INFO] training autoencoder...')
H=autoencoder2.fit(
    trainData,
    validation_data=valData,
    epochs=EPOCHS,
    callbacks=[DisplayCallback(), RecordBestLossCallback(), model_checkpoint_callback]
)

print('[INFO] loading best weights and cleanup')
autoencoder2.load_weights(checkpoint_filepath)
!rm -rf {checkpoint_filepath}/*

print('[INFO] Saving model...')
autoencoder.save(modelSavePath2)

print('Final Prediction:')
show_predictions(autoencoder2, visImages[:3], 3)
show_predictions(autoencoder2, visImages[3:], 3)

print(f'Best val loss: {best_loss} at epoch {best_epoch}')